<a href="https://colab.research.google.com/github/harshelke180502/Big_Data_Mining-CS483-Harsh_Shelke/blob/main/Harsh_Shelke_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 2
## Frequent Pattern Mining in Spark

In [ ]:
!pip install -U pyspark

import pyspark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [ ]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [ ]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [ ]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [ ]:
''' 2 lines of code expected '''
# YOUR CODE HERE
data = orders.join(products, on="product_id", how="inner")
data.show(5)


+----------+--------+-----------------+---------+--------------------+--------+-------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|aisle_id|department_id|
+----------+--------+-----------------+---------+--------------------+--------+-------------+
|     49302|       1|                1|        1|    Bulgarian Yogurt|     120|           16|
|     11109|       1|                2|        1|Organic 4% Milk F...|     108|           16|
|     10246|       1|                3|        0|Organic Celery He...|      83|            4|
|     49683|       1|                4|        0|      Cucumber Kirby|      83|            4|
|     43633|       1|                5|        1|Lightly Smoked Sa...|      95|           15|
+----------+--------+-----------------+---------+--------------------+--------+-------------+
only showing top 5 rows


In [ ]:
from pyspark.sql.functions import collect_list

baskets = data.groupBy("order_id").agg(collect_list("product_name").alias("items"))

baskets.show(5, truncate=False)

+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                                                                                                                                                                                        |
+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1       |[Bulgarian Yogurt, Organic 4% Milk Fat Whole Milk C

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [ ]:
''' 3 lines of code expected '''
# YOUR CODE HERE

from pyspark.ml.fpm import FPGrowth

fp = FPGrowth(itemsCol="items",
              minSupport=0.01,
              minConfidence=0.5)

model = fp.fit(baskets)


Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
frequent_itemsets = model.freqItemsets
frequent_itemsets.show(10, truncate=False)



+----------------------------------------------+-----+
|items                                         |freq |
+----------------------------------------------+-----+
|[Banana]                                      |18726|
|[Organic Strawberries]                        |10894|
|[Organic Strawberries, Banana]                |2174 |
|[Organic Strawberries, Bag of Organic Bananas]|3074 |
|[Large Lemon]                                 |8135 |
|[Large Lemon, Banana]                         |2158 |
|[Organic Hass Avocado]                        |7293 |
|[Organic Hass Avocado, Bag of Organic Bananas]|2420 |
|[Organic Hass Avocado, Organic Strawberries]  |1539 |
|[Limes]                                       |6033 |
+----------------------------------------------+-----+
only showing top 10 rows


In [ ]:
print("Number of frequent itemsets:",
      frequent_itemsets.count())


Number of frequent itemsets: 120


In [ ]:
association_rules = model.associationRules
association_rules.show(10, truncate=False)


+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



In [ ]:
print("Number of association rules:",
      association_rules.count())


Number of association rules: 0


Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE

fp_new = FPGrowth(
    itemsCol="items",
    minSupport=0.001,
    minConfidence=0.5
)

model_new = fp_new.fit(baskets)


frequent_itemsets_new = model_new.freqItemsets
association_rules_new = model_new.associationRules


print("Number of frequent itemsets (support=0.001):",
      frequent_itemsets_new.count())

print("Number of association rules (support=0.001):",
      association_rules_new.count())





Number of frequent itemsets (support=0.001): 4444
Number of association rules (support=0.001): 11


Q3: After retraining the FP-Growth model with the minSupport parameter set to 0.001, report the first 10 frequent itemsets ordered by frequency, and the first 10 association rules ordered by confidence.

In [ ]:
# YOUR CODE HERE
frequent_itemsets_new.orderBy("freq", ascending=False).show(10, truncate=False)


association_rules_new.orderBy("confidence", ascending=False).show(10, truncate=False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
|[Organic Avocado]       |7409 |
|[Organic Hass Avocado]  |7293 |
|[Strawberries]          |6494 |
|[Limes]                 |6033 |
|[Organic Raspberries]   |5546 |
+------------------------+-----+
only showing top 10 rows
+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|antecedent                                                       |consequent              |confidence        |lift              |support              |
+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|[Organic Raspberries, Organic Hass Avocado, Organic S

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!